In [156]:
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical 
import numpy as np 
import pandas as pd

from sklearn.preprocessing import minmax_scale 
from sklearn.model_selection import train_test_split

from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import Adam
import itertools

In [157]:
file_directory='../Preprocess/data_after/'
#opposite과 my_team에 대한 변수를 두어 그 값이 1이면 내 팀, 0이면 다른 팀 인 방식으로 구현
#테스트 데이터는 match.csv의 result attribute를 사용
#train_data에 사용되는 데이터는 player1.csv의 선수의 티어 값을 이용
(X_train, y_train), (X_test, y_test) = mnist.load_data() 
print(X_train.shape)
print(y_train)

(60000, 28, 28)
[5 0 4 ... 5 6 8]


In [158]:

#player1.csv의 파일을 읽어서 각 팀별 플레이어 티어 평균값을 추출
data=pd.read_csv("{0}player1.csv".format(file_directory))
tier = data.loc[:,['Team','Tier']]

team = data.sort_values('Team' ,ascending=True)
team = team.loc[:,'Team'].drop_duplicates()
team = team.to_list()
print(team)
AvgTier = tier.groupby('Team').mean()
AvgTier2 = np.random.rand(AvgTier.shape[0])
AvgTier2 = AvgTier2.reshape((31,1))*5
print(AvgTier2)


AvgTier = AvgTier.to_numpy()

Avg = np.zeros((31,2))

AvgTier = AvgTier.reshape(31)
AvgTier2 = AvgTier2.reshape(31)

Avg[:,0] = AvgTier
Avg[:,1] = AvgTier2


print(Avg)

X_train = Avg

y_train = np.zeros(31)


for i in range(AvgTier.shape[0]):
    if AvgTier[i]<AvgTier2[i]:
        y_train[i] = 1
    else:
        y_train[i] = 0

y_train = to_categorical(y_train)
print(y_train)

['CJ Intas', 'DRK', 'DWG KOE', 'Efriice Friics', 'Fridot BROUN', 'Gin.G', 'Groffon', 'Henwhe Lofi Ispurts', 'ISC Ivir', 'Ivir8 Wonnirs', 'Jon Eor Griin Wongs', 'KSV iSpurts', 'KT Rulstir', 'Kongzuni DregunX', 'Kungduu Munstir', 'Loov SENDBUX', 'Lungzha Gemong', 'MBP', 'NeJon i-mFori', 'Nungshom RidFurci', 'Oncridobli Morecli', 'RUX Togirs', 'Ribils Enerchy', 'SBINA Kurie', 'SK Tilicum T1', 'Semsang Gelexy', 'SiulHeiUni Pronci', 'T2', 'Tiem Dynemocs', 'bbq Ulovirs', 'i-mFori']
[[3.22074254]
 [1.56349151]
 [2.97271227]
 [3.64801681]
 [3.88329311]
 [0.62281611]
 [0.55661549]
 [1.81260071]
 [4.39421978]
 [4.5943412 ]
 [0.79289848]
 [3.08222756]
 [0.47133076]
 [2.67043075]
 [2.14032862]
 [1.00068851]
 [3.1081158 ]
 [3.99537956]
 [3.14890571]
 [4.87523082]
 [2.26633835]
 [1.69583938]
 [3.5636273 ]
 [1.40475543]
 [0.26325535]
 [3.52701118]
 [1.68994091]
 [3.93238393]
 [1.46934208]
 [3.96718956]
 [0.54309659]]
[[2.57142857 3.22074254]
 [1.88235294 1.56349151]
 [1.94117647 2.97271227]
 [2.51282

In [159]:
#match.csv 파일을 읽어서 팀의 승패 결과를 test 데이터로 활용
data=pd.read_csv("{0}match.csv".format(file_directory))

X_test = data.loc[:,['side','team','result']]
y_test = np.zeros(31)

for i in range(len(team)):
    for idx in X_test.index:
        if team[i]==X_test.loc[idx,'team']:
            y_test[i] = X_test.loc[idx,'result']
            break
y_test = to_categorical(y_test)

print(y_test)

[[0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]]


In [160]:
TestTier = np.random.rand(AvgTier.shape[0])
TestTier = TestTier.reshape(AvgTier.shape[0])*5


TestTier2 = np.random.rand(AvgTier.shape[0])
TestTier2 = TestTier2.reshape(AvgTier.shape[0])*5

X_test = np.zeros((AvgTier.shape[0],2))

X_test[:,0] = TestTier
X_test[:,1] = TestTier2


for i in range(X_test.shape[0]):
    if y_test[i,0]==0:
        if X_test[i,0] <X_test[i,1]:
            temp = X_test[i,0]
            X_test[i,0] = X_test[i,1]
            X_test[i,1] = temp
            
    elif y_test[i,0]==1:
        if X_test[i,0]>X_test[i,1]:
            temp = X_test[i,0]
            X_test[i,0] = X_test[i,1]
            X_test[i,1] = temp

In [161]:
print(X_train.shape)
print(y_train.shape)
print(X_test)
print(y_test)
num_of_train_samples = X_train.shape[0] 
width = X_train.shape[1]    
print(width)
print(num_of_train_samples)


(31, 2)
(31, 2)
[[4.10289189 1.37496555]
 [0.03310653 0.25244514]
 [4.83340188 4.02054654]
 [0.44309406 3.11022354]
 [3.3139184  4.86794181]
 [0.5554724  4.59818955]
 [3.61678135 4.15161853]
 [1.52682272 0.23178315]
 [2.80870572 1.81528233]
 [1.77626408 0.397231  ]
 [3.6696364  1.46295166]
 [4.74654398 1.47506949]
 [0.32042281 2.63494247]
 [2.50433556 0.47341888]
 [1.21360825 4.89836395]
 [0.75808336 0.72785191]
 [4.15450004 1.79223877]
 [0.62251939 1.39655404]
 [2.51409116 0.23106483]
 [3.27360579 1.89613324]
 [2.81145504 2.89287996]
 [4.69728824 0.19817098]
 [3.34040795 4.07803739]
 [2.47040547 2.79858231]
 [0.99329781 2.40855237]
 [0.1745455  3.54544494]
 [1.47770988 3.54914007]
 [0.9981285  3.83682255]
 [4.94873557 0.65509431]
 [1.98912608 3.52269709]
 [2.95173297 4.11183953]]
[[0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 

In [162]:
num_of_test_samples = X_test.shape[0] 
print(num_of_test_samples)

31


In [193]:
model = Sequential()

In [194]:
#input layer
model.add(Dense(32, input_shape = (31,2), kernel_initializer='glorot_uniform', activation='relu'))
model.add(Dropout(0.3)) 
for i in range(80):
    model.add(Dense(16, kernel_initializer='glorot_uniform', activation='relu'))
    model.add(Dropout(0.3))


In [195]:
print(X_train)
print(y_train)

[[2.57142857 3.22074254]
 [1.88235294 1.56349151]
 [1.94117647 2.97271227]
 [2.51282051 3.64801681]
 [3.11111111 3.88329311]
 [2.47727273 0.62281611]
 [2.12       0.55661549]
 [2.76       1.81260071]
 [3.         4.39421978]
 [3.875      4.5943412 ]
 [3.08974359 0.79289848]
 [3.16666667 3.08222756]
 [2.46067416 0.47133076]
 [1.92       2.67043075]
 [3.55555556 2.14032862]
 [2.75675676 1.00068851]
 [2.44827586 3.1081158 ]
 [3.06666667 3.99537956]
 [2.5625     3.14890571]
 [2.6        4.87523082]
 [3.35294118 2.26633835]
 [2.46666667 1.69583938]
 [3.         3.5636273 ]
 [3.73333333 1.40475543]
 [2.1369863  0.26325535]
 [2.55       3.52701118]
 [3.47058824 1.68994091]
 [1.96       3.93238393]
 [2.8        1.46934208]
 [3.125      3.96718956]
 [3.83333333 0.54309659]]
[[0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]

In [196]:
model.add(Dense(2, activation='softmax'))#soft_max 

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [198]:
training_epochs = 30 
batch_size = 100 
model.fit(X_train, y_train, epochs=training_epochs, batch_size=batch_size)


Epoch 1/30
1/1 [==============================] - 0s 5ms/step - loss: 0.6928 - accuracy: 0.5161
Epoch 2/30
1/1 [==============================] - 0s 5ms/step - loss: 0.6932 - accuracy: 0.5161
Epoch 3/30
1/1 [==============================] - 0s 6ms/step - loss: 0.6933 - accuracy: 0.5161
Epoch 4/30
1/1 [==============================] - 0s 6ms/step - loss: 0.6934 - accuracy: 0.5161
Epoch 5/30
1/1 [==============================] - 0s 5ms/step - loss: 0.6927 - accuracy: 0.5161
Epoch 6/30
1/1 [==============================] - 0s 5ms/step - loss: 0.6930 - accuracy: 0.5161
Epoch 7/30
1/1 [==============================] - 0s 5ms/step - loss: 0.6929 - accuracy: 0.5161
Epoch 8/30
1/1 [==============================] - 0s 5ms/step - loss: 0.6929 - accuracy: 0.5161
Epoch 9/30
1/1 [==============================] - 0s 5ms/step - loss: 0.6930 - accuracy: 0.5161
Epoch 10/30
1/1 [==============================] - 0s 6ms/step - loss: 0.6930 - accuracy: 0.5161
Epoch 11/30
1/1 [======================

In [199]:
evaluation = model.evaluate(X_test, y_test, batch_size=batch_size) 
print('Accuracy: ' + str(evaluation[1]))


1/1 [==============================] - 0s 329ms/step - loss: 0.6914 - accuracy: 0.5484
Accuracy: 0.5483871102333069
